Описание: Необходимо создать систему, использующую
паттерны Наблюдатель и Состояние. Пользователь может
находиться в разных состояниях (например, "Онлайн", "Отошел",
"Не беспокоить"), и система должна уведомлять наблюдателей
о смене состояния.


Требования4
)] Паттерн Наблюдатель(
& Создайте интерфейс Observer для уведомлений]
& Реализуйте класс Subject, который будет уведомлять
наблюдателей о смене состояния пользователя]
] Паттерн Состояние(
& Реализуйте интерфейс состояния с методом handle()]
& Создайте классы состояний: OnlineState, AwayState,
DoNotDisturbState]
] Логика работы(
& При смене состояния пользователя уведомляйте всех
наблюдателей о текущем состоянии]
& Поведение системы должно зависеть от состояния
(например, в DoNotDisturbState уведомления не
приходят)]
M] Пример(
& Продемонстрируйте работу системы, где несколько
наблюдателей следят за состоянием пользователя.


Дополнительно4
& Проект должен быть легко расширяемым для добавления
новых состояний и наблюдателей.

In [2]:
#Observer
from abc import ABC, abstractmethod

class Observer(ABC):
    @abstractmethod
    def update(self, user):
        """Метод для получения уведомлений об изменении состояния"""
        pass

#Пользователь
class UserSubject:
    def __init__(self, name):
        self.name = name
        self._observers = []
        self._state = None
    
    def attach(self, observer):
        if observer not in self._observers:
            self._observers.append(observer)
    
    def detach(self, observer):
        if observer in self._observers:
            self._observers.remove(observer)
    
    def notify(self):
        """Уведомление всех наблюдателей, кроме случаев 'Не беспокоить'"""
        if not isinstance(self._state, DoNotDisturbState):
            for observer in self._observers:
                observer.update(self)
    
    @property
    def state(self):
        return self._state
    
    @state.setter
    def state(self, state):
        self._state = state
        self.notify()
    
    def __str__(self):
        return f"{self.name} ({self.state})"
    
#Состояние
class UserState(ABC):
    @abstractmethod
    def handle(self):
        """Метод для обработки состояния"""
        pass
    
    def __str__(self):
        return self.__class__.__name__.replace("State", "")
    
class OnlineState(UserState):
    def handle(self):
        print("Пользователь онлайн и доступен для общения")

class AwayState(UserState):
    def handle(self):
        print("Пользователь отошел, но может ответить")

class DoNotDisturbState(UserState):
    def handle(self):
        print("Пользователь не хочет получать уведомления")

#наблюдатели
class NotificationObserver(Observer):
    def update(self, user):
        print(f"[Уведомление] Статус {user.name} изменен на: {user.state}")
        user.state.handle()

class LoggingObserver(Observer):
    def update(self, user):
        print(f"[Лог] {user.name} сменил статус на {user.state} в {datetime.now().strftime('%H:%M:%S')}")

class FriendObserver(Observer):
    def __init__(self, name):
        self.name = name
    
    def update(self, user):
        if isinstance(user.state, OnlineState):
            print(f"[Друг {self.name}] Ура! {user.name} теперь онлайн!")
        elif isinstance(user.state, AwayState):
            print(f"[Друг {self.name}] {user.name} отошел...")

#использование системы 

from datetime import datetime

def main():
    # Создаем пользователя
    user = UserSubject("Иван Иванов")
    
    # Создаем наблюдателей
    notification = NotificationObserver()
    logger = LoggingObserver()
    friend1 = FriendObserver("Петя")
    friend2 = FriendObserver("Маша")
    
    # Подписываем наблюдателей
    user.attach(notification)
    user.attach(logger)
    user.attach(friend1)
    user.attach(friend2)
    
    # Меняем состояния пользователя
    print("\n--- Устанавливаем статус 'Онлайн' ---")
    user.state = OnlineState()
    
    print("\n--- Устанавливаем статус 'Отошел' ---")
    user.state = AwayState()
    
    print("\n--- Устанавливаем статус 'Не беспокоить' ---")
    user.state = DoNotDisturbState()  # Наблюдатели не получат уведомление
    
    print("\n--- Устанавливаем статус 'Онлайн' снова ---")
    user.state = OnlineState()
    
    # Отписываем одного наблюдателя
    print("\n--- Отписываем друга Петю ---")
    user.detach(friend1)
    
    print("\n--- Устанавливаем статус 'Отошел' ---")
    user.state = AwayState()

if __name__ == "__main__":
    main()
    


--- Устанавливаем статус 'Онлайн' ---
[Уведомление] Статус Иван Иванов изменен на: Online
Пользователь онлайн и доступен для общения
[Лог] Иван Иванов сменил статус на Online в 12:06:06
[Друг Петя] Ура! Иван Иванов теперь онлайн!
[Друг Маша] Ура! Иван Иванов теперь онлайн!

--- Устанавливаем статус 'Отошел' ---
[Уведомление] Статус Иван Иванов изменен на: Away
Пользователь отошел, но может ответить
[Лог] Иван Иванов сменил статус на Away в 12:06:06
[Друг Петя] Иван Иванов отошел...
[Друг Маша] Иван Иванов отошел...

--- Устанавливаем статус 'Не беспокоить' ---

--- Устанавливаем статус 'Онлайн' снова ---
[Уведомление] Статус Иван Иванов изменен на: Online
Пользователь онлайн и доступен для общения
[Лог] Иван Иванов сменил статус на Online в 12:06:06
[Друг Петя] Ура! Иван Иванов теперь онлайн!
[Друг Маша] Ура! Иван Иванов теперь онлайн!

--- Отписываем друга Петю ---

--- Устанавливаем статус 'Отошел' ---
[Уведомление] Статус Иван Иванов изменен на: Away
Пользователь отошел, но может о